# Foreground LCIA

This notebook calculates the direct impacts using emission factors for onsite emissions. These are regionalized for eutrophication impacts.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#In this cell the scenario is chosen. This can either be defined here or in the RUN_ALL.ipynb
#If defined here unmark the scen = line and select the scenario you will run.
#If defined in RUN_ALL.ipynb unmark the %store -r scen line.
#Scenario names are:
#S0_MinFert > N,P,K demand met by mineral fertilizers only
#S0_struvite_P > P demand met by struvite, N demand met by struvite and mineral fertilizer and K demand met by mineral fertilizer
#S0_compost > N,P,K supplied by compost produced in the AMB, remaining N,P,K demand met by mineral fertilizer
#S0_Ammon_salts > N supplied from recovered ammonium salts and from mineral fertilizers, P from struvite and K from mineral fertilizer
scen = 'S0_MinFert'
#%store -r scen

In [ ]:
print(scen)

In [ ]:
%store -r d_conv_emiss
%store -r URBAG_map

## Emission factors

In [ ]:
#Emission factors for direct emissions according to ReCiPe v1.13 (H) + Regionalized eutrophication CFs for Besos and Llobregat
CFs_foreground_impacts = pd.read_excel("inputs/CFs_foreground_impacts.xlsx")
CFs_foreground_impacts

In [ ]:
CFs_foreground_impacts.keys()

In [ ]:
CO2_CFs   = CFs_foreground_impacts[(CFs_foreground_impacts.env_flow_foreground == 'carbon_dioxide_to_air')]
CH4_CFs   = CFs_foreground_impacts[(CFs_foreground_impacts.env_flow_foreground == 'methane_to_air')]
N2O_CFs   = CFs_foreground_impacts[(CFs_foreground_impacts.env_flow_foreground == 'dinitrogen_monoxide_to_air')]
NO3_CFs   = CFs_foreground_impacts[(CFs_foreground_impacts.env_flow_foreground == 'nitrate_to_water')]
N_CFs     = CFs_foreground_impacts[(CFs_foreground_impacts.env_flow_foreground == 'nitrogen_to_water')]
PO43_CFs  = CFs_foreground_impacts[(CFs_foreground_impacts.env_flow_foreground == 'phosphate_to_water')]
P_CFs     = CFs_foreground_impacts[(CFs_foreground_impacts.env_flow_foreground == 'phosphorus_to_water')]
water_CFs = CFs_foreground_impacts[(CFs_foreground_impacts.env_flow_foreground == 'water')]

### LCIA Methods: Family and version, Name, Unit, # of CFsS


- IPCC 2021, climate change, global warming potential (GWP100)	kg CO2-Eq	226  >>> N2O, CO2, CH4 to air
- ReCiPe 2016 v1.03, midpoint (H), eutrophication: marine, marine eutrophication potential (MEP)	kg N-Eq	22 >>> Nitrate to water
- ReCiPe 2016 v1.03, midpoint (H), eutrophication: freshwater, freshwater eutrophication potential (FEP)	kg P-Eq	13 >>> Phosphate to water
- ReCiPe 2016 v1.03, midpoint (H), water use, water consumption potential (WCP) m3 of water consumed/m3 water extracted 5 >>> Water Extraction 
- CML v4.8 2016, material resources: metals/minerals, abiotic depletion potential (ADP): elements (ultimate reserves)	kg Sb-Eq	129 >> NO DIRECT EMISSIONS

#### WCP - Midpoint level ReCiPe

Water extraction, is the withdrawal of water from surface water bodies or the abstraction of groundwater from aquifer. It is the total amount of water withdrawn, irrespective of
return flows to the water bodies or water use efficiencies.
Water consumption, on the other hand is the amount of water that the watershed of origin is losing.
- CF is:
    - 1 if inventory is in water consumption
    - water requirement ratio (Spain 0,6; world-agriculture 0.44) if inventory is in water extraction
      

In [ ]:
d_conv_emiss.keys()

In [ ]:
URBAG_map.keys().values

In [ ]:
pd.unique(URBAG_map['CUENCA'])

In [ ]:
URBAG_map["Cuencas"] = 0

In [ ]:
URBAG_map["Cuencas"][URBAG_map['CUENCA'].str.contains("Maresme")] = "Besos"
URBAG_map["Cuencas"][URBAG_map['CUENCA'].str.contains("Besos")]   = "Besos"
URBAG_map["Cuencas"][URBAG_map['CUENCA'].str.contains("Llobregat")]       = "Llobregat"
URBAG_map["Cuencas"][URBAG_map['CUENCA'].str.contains("CiutatBarcelona")] = "Llobregat"
URBAG_map["Cuencas"][URBAG_map['CUENCA'].str.contains("Garraf")]          = "Llobregat"

In [ ]:
pd.unique(URBAG_map["Cuencas"])

### Direct impacts

#### N2O foreground impacts

In [ ]:
#kg CO2eq per ha yr
URBAG_map['GWP100_IPCC_N2O'] = d_conv_emiss['N2O_total_air'].values * N2O_CFs.loc[N2O_CFs['impact_cat']=='climate_IPCC']['CF_recipe'].values

#### CO2 foreground impacts

In [ ]:
#kg CO2eq per ha yr
URBAG_map['GWP100_IPCC_CO2'] = d_conv_emiss['CO2_total_air'].values * CO2_CFs.loc[CO2_CFs['impact_cat']=='climate_IPCC']['CF_recipe'].values

#### CH4 foreground impacts

In [ ]:
#kg CO2eq per ha yr
URBAG_map['GWP100_IPCC_CH4'] = d_conv_emiss['CH4_burnwaste_air'].values * CH4_CFs.loc[CH4_CFs['impact_cat']=='climate_IPCC']['CF_recipe'].values

#### NO3- foreground impacts

In [ ]:
#kg Neq per ha yr
URBAG_map['ME_NO3'] = d_conv_emiss['NO3_total'].values * NO3_CFs.loc[NO3_CFs['impact_cat']=='marine_eutroph']['CF_recipe'].values

In [ ]:
#kg Neq per ha yr
cond1 = URBAG_map['Cuencas'] == 'Llobregat' 
cond2 = URBAG_map['Cuencas'] == 'Besos'

#For plots in condition 1: Midpoint regionalised impacts are
URBAG_map.loc[URBAG_map.index[cond1],'ME_NO3_Reg'] = d_conv_emiss.loc[d_conv_emiss.index[cond1],'NO3_total'].values * NO3_CFs.loc[NO3_CFs['impact_cat']=='marine_eutroph']['CF_Llobregat_midpoint'].values

#For plots in condition 2: Midpoint regionalised impacts are
URBAG_map.loc[URBAG_map.index[cond2],'ME_NO3_Reg'] = d_conv_emiss.loc[d_conv_emiss.index[cond2],'NO3_total'].values * NO3_CFs.loc[NO3_CFs['impact_cat']=='marine_eutroph']['CF_Besòs_midpoint'].values

#### PO43- foreground impacts

In [ ]:
#kg Peq per ha yr
URBAG_map['FE_PO43'] = d_conv_emiss['PO43_runoff_water_total'].values * PO43_CFs.loc[PO43_CFs['impact_cat']=='Freshwater_eutroph']['CF_recipe'].values

In [ ]:
#kg Peq per ha yr
cond1 = URBAG_map['Cuencas'] == 'Llobregat' 
cond2 = URBAG_map['Cuencas'] == 'Besos'

#For plots in condition 1: Midpoint regionalised impacts are
URBAG_map.loc[URBAG_map.index[cond1],'FE_PO43_Reg'] = d_conv_emiss.loc[d_conv_emiss.index[cond1],'PO43_runoff_water_total'].values * PO43_CFs.loc[PO43_CFs['impact_cat']=='Freshwater_eutroph']['CF_Llobregat_midpoint'].values

#For plots in condition 2: Midpoint regionalised impacts are
URBAG_map.loc[URBAG_map.index[cond2],'FE_PO43_Reg'] = d_conv_emiss.loc[d_conv_emiss.index[cond2],'PO43_runoff_water_total'].values * PO43_CFs.loc[PO43_CFs['impact_cat']=='Freshwater_eutroph']['CF_Besòs_midpoint'].values

#### Water consumption

In [ ]:
#m3 water consumed/ ha yr
URBAG_map['WC'] = URBAG_map['WE_m3_ha_yr'] * water_CFs.loc[water_CFs['impact_cat']=='water_consumption_potential']['CF_recipe'].values

In [ ]:
#calculate total water consumption AMB m3/yr
np.sum(URBAG_map['WC']*URBAG_map['Area']*(1/10000))

In [ ]:
foreground_impacts = pd.DataFrame(URBAG_map)

In [ ]:
foreground_impacts.keys().values

In [ ]:
pd.unique(foreground_impacts['FE_PO43_Reg'])

In [ ]:
foreground_impacts = foreground_impacts.drop(['TIPO_RIEGO', 'EFIC_RIEGO', 'TIPO_DIST', 'EFIC_DISTR', 'DUN_GRUP',
       'DUN_CULTIU', 'Voronoi_1', 'TF', 'REG_CREAF', 'REG_CAT', 'TF_REG',
       'CAT_NIV_5', 'ORD_NIV_5', 'CAT_NIV_4', 'ORD_NIV_4', 'CAT_NIV_3',
       'ORD_NIV_3', 'CAT_NIV_2', 'ORD_NIV_2', 'CAT_NIV_1F', 'ORD_NIV_1F',
       'CAT_NIV_1', 'ORD_NIV_1', '01_ETO', '02_ETO', '03_ETO', '04_ETO',
       '05_ETO', '06_ETO', '07_ETO', '08_ETO', '09_ETO', '10_ETO',
       '11_ETO', '12_ETO', '01_PPT', '02_PPT', '03_PPT', '04_PPT',
       '05_PPT', '06_PPT', '07_PPT', '08_PPT', '09_PPT', '10_PPT',
       '11_PPT', '12_PPT', 'ID_Voronoi', 'Municipi', 'Codi_munic',
        'Comarca', 'Codi_comar', 'ID_CUENCA', 'CUENCA',
       'ID_SUBCUEN', 'N_SUBCUENC', 'SUBCUENCAS',
       'yield_kgHA', 'kgP2O5/ha', 'kgK2O/ha', 'kgN/ha', 'EF_Tier2',
       'id_final', 'Kc_b_Kc1', 'Kc_b_Kc2', 'Kc_b_Kc3', 'Kc_b_Kc4',
       'Kc_b_Kc5', 'Kc_b_Kc6', 'Kc_b_Kc7', 'Kc_b_Kc8', 'Kc_b_Kc9',
       'Kc_b_Kc10', 'Kc_b_Kc11', 'Kc_b_Kc12', 'N_T', 'P_T', 'K_T',
       'Voronoi_ag', 'Vor_ag_Num', 'FoodTons_N', '01_CWR', '02_CWR',
       '03_CWR', '04_CWR', '05_CWR', '06_CWR', '07_CWR', '08_CWR',
       '09_CWR', '10_CWR', '11_CWR', '12_CWR', '01_PPTe', '02_PPTe',
       '03_PPTe', '04_PPTe', '05_PPTe', '06_PPTe', '07_PPTe', '08_PPTe',
       '09_PPTe', '10_PPTe', '11_PPTe', '12_PPTe', '01_ID', '02_ID',
       '03_ID', '04_ID', '05_ID', '06_ID', '07_ID', '08_ID', '09_ID',
       '10_ID', '11_ID', '12_ID', 'LOSSES', '01_WE', '02_WE', '03_WE',
       '04_WE', '05_WE', '06_WE', '07_WE', '08_WE', '09_WE', '10_WE',
       '11_WE', '12_WE', 'WE_m3_ha_yr', 'Manure', 'Manure_N',
       'Manure_P2O5', 'Agriwaste_toField', 'Agriwaste_burned',
       'N_agriwaste_to_field', 'Cuencas'], axis=1)

In [ ]:
foreground_impacts.keys()

In [ ]:
#Foreground impacts ME kg 
d2 = {'ME_NO3' : foreground_impacts['ME_NO3'],
      'ME_NO3_Reg': foreground_impacts['ME_NO3_Reg'],
      }
df2 = pd.DataFrame(data=d2)
df2.plot(kind="box", showfliers=False)
plt.title('ME - kg Neq per ha yr')

In [ ]:
#Foreground impacts GWP
d3 = {'GWP100_IPCC_N2O': foreground_impacts['GWP100_IPCC_N2O'], 
     'GWP100_IPCC_CO2': foreground_impacts['GWP100_IPCC_CO2'],
     'GWP100_IPCC_CH4': foreground_impacts['GWP100_IPCC_CH4'],
    }
df3 = pd.DataFrame(data=d3)
df3.plot(kind="box", showfliers=False)
plt.title('GWP - kg CO2eq per ha yr')

In [ ]:
#Foreground impacts FE
d4 = {
      'FE_PO43': foreground_impacts['FE_PO43'],
      'FE_PO43_Reg': foreground_impacts['FE_PO43_Reg'],
    }
df4 = pd.DataFrame(data=d4)
df4.plot(kind="box", showfliers=False)
plt.title('FE - kg Peq per ha yr')

In [ ]:
#Foreground impacts Irrigation
d5 = {
      'WC': foreground_impacts['WC'],
    }
df5 = pd.DataFrame(data=d5)
df5.plot(kind="box", showfliers=False)
plt.title('WC - m3 per ha yr')

In [ ]:
#Select only impacts you want to have in the figures
foreground_impacts_1 = pd.melt(foreground_impacts, id_vars=[ 'Fid',"Area"], value_vars=[
        'GWP100_IPCC_N2O', 
        'GWP100_IPCC_CO2', 
        'GWP100_IPCC_CH4', 
        'ME_NO3',
        'ME_NO3_Reg', 
        'FE_PO43', 
        'FE_PO43_Reg', 
        'WC'
        ],var_name='impact_emission')

In [ ]:
pd.unique(foreground_impacts_1['impact_emission'])

In [ ]:
foreground_impacts_1.loc[(foreground_impacts_1["Fid"]==2544)
                         &(foreground_impacts_1["impact_emission"]=="WC")]

In [ ]:
imp_emiss_to_impact = {'GWP100_IPCC_N2O':['GWP100'],
                        'GWP100_IPCC_CO2':['GWP100'],
                        'GWP100_IPCC_CH4':['GWP100'],
                        'ME_NO3':['ME'],
                        'ME_NO3_Reg':['ME_Reg'],
                        'FE_PO43':['FE'],
                        'FE_PO43_Reg':['FE_Reg'],
                        'WC':['WC']
                          }

In [ ]:
imp_emiss_to_emiss = {'ME_NO3':['Nitrate'],
                        'ME_NO3_Reg':['Nitrate'],
                        'GWP100_IPCC_N2O':['N2O'],
                        'GWP100_IPCC_CO2':['CO2'],
                        'GWP100_IPCC_CH4':['CH4'],
                        'FE_PO43':['Phosphate'],
                        'FE_PO43_Reg':['Phosphate'],
                        'WC':['Irrigation']
                         }

In [ ]:
imp_emiss = {}
for key, impacts in imp_emiss_to_impact.items():
    
    for x in impacts:
        imp_emiss[key] = x

foreground_impacts_1['imp_cat'] = foreground_impacts_1['impact_emission'].map(imp_emiss)

pd.unique(foreground_impacts_1['imp_cat'])

In [ ]:
imp_emiss_subs = {}
for key, subs in imp_emiss_to_emiss.items():
    
    for x in subs:
        imp_emiss_subs[key] = x

foreground_impacts_1['product'] = foreground_impacts_1['impact_emission'].map(imp_emiss_subs)

pd.unique(foreground_impacts_1['product'])

In [ ]:
foreground_impacts_1 = foreground_impacts_1.drop('impact_emission', axis=1)

In [ ]:
foreground_impacts_1.keys()

In [ ]:
foreground_impacts_1.rename(columns={'value': 'imp_per_ha'}, inplace=True)

In [ ]:
#Adding total impacts per year
#Total impact per parcel in kg impact units/ha*yr and for water in m3/ha yr
#!!!!!!conveting kg to tons>> GWP = Ton CO2eq; MEP= Ton Neq; FEP= Ton P eq; and WD = from m3 to 1000 m3!!!!!
#Final units for impacts are ton impact unit/yr and 1000 m3/yr
#Adding foreground label
#Adding scenario scen
#Adding no database as this are direct emissions
foreground_impacts_1['imp_yr']=foreground_impacts_1['imp_per_ha']*foreground_impacts_1['Area']*(1/10000)/1000
foreground_impacts_1['LCIA']='foreground'
foreground_impacts_1['Scenario']=scen
foreground_impacts_1['database']='ei391'

In [ ]:
foreground_impacts_1.keys()

In [ ]:
#imp_per_ha in kg impact per ha year
#imp_yr in ton per year
data_foreground = foreground_impacts_1

In [ ]:
data_foreground.loc[data_foreground['Fid']==2544]

In [ ]:
data_foreground_GWP = data_foreground[(data_foreground['imp_cat']=='GWP100')]
len(data_foreground_GWP)

In [ ]:
#number of plots times substances emitted per category (3 for GWP)
4581*3

In [ ]:
print("Total GMP foreground impacts in tons CO2eq/yr are " + str(data_foreground_GWP.imp_yr.sum()))

In [ ]:
data_foreground_ME = data_foreground[(data_foreground['imp_cat']=='ME')]
len(data_foreground_ME)

In [ ]:
data_foreground_ME_Reg = data_foreground[(data_foreground['imp_cat']=='ME_Reg')]
len(data_foreground_ME_Reg)

In [ ]:
data_foreground_ME_Reg.imp_yr.sum()

In [ ]:
print("Total ME_Reg foreground impacts in tons Neq/yr are " + str(data_foreground_ME_Reg.imp_yr.sum()))

In [ ]:
data_foreground_FE = data_foreground[(data_foreground['imp_cat']=='FE')]
len(data_foreground_FE)

In [ ]:
data_foreground_FE_Reg = data_foreground[(data_foreground['imp_cat']=='FE_Reg')]
len(data_foreground_FE_Reg)

In [ ]:
print("Total FE_Reg foreground impacts in tons Peq/yr are " + str(data_foreground_FE_Reg.imp_yr.sum()))

In [ ]:
data_foreground_WC = data_foreground[(data_foreground['imp_cat']=='WC')]
len(data_foreground_WC)

In [ ]:
print("Total WC foreground impacts in *1000 m3/yr are " + str(data_foreground_WC.imp_yr.sum()))

In [ ]:
#Databases to be included as foreground, for visualization purposes. This does not matter for calculations
databases = ['ei391', 
#             'ei38_SSP2-Base_30', 
#             'ei38_SSP2-Base_40',
             'ei391_Image_SSP2_Base_50', 
#             'ei38_SSP2-RCP2.6_30', 
#             'ei38_SSP2-RCP2.6_40',
             'ei391_Image_SSP2_RCP26_50'
            ]

In [ ]:
#copying the foreground impacts the same amount of background databases (foreground is the same in all scenarios while background changes)
data_foreground = pd.concat([data_foreground.replace('ei391',db) for db in databases],ignore_index=True)
data_foreground

In [ ]:
#Total impact in ton impact unit or *1000 m3/yr for the whole AMB, per product and database and scenario
data_foreground_1 = data_foreground.groupby(["LCIA",'database',"imp_cat", "Scenario", "product"])["imp_yr"].aggregate("sum").reset_index()
data_foreground_1

## Regionalized Eutrophication Impacts

In [ ]:
URBAG_map.keys().values

In [ ]:
regional_impacts = URBAG_map.drop(['TIPO_RIEGO', 'EFIC_RIEGO', 'TIPO_DIST', 'EFIC_DISTR', 'TF', 'REG_CREAF', 'REG_CAT', 'TF_REG',
       'CAT_NIV_4', 'ORD_NIV_4', 'CAT_NIV_3',
       'ORD_NIV_3', 'CAT_NIV_2', 'ORD_NIV_2', 'CAT_NIV_1F', 'ORD_NIV_1F',
       'CAT_NIV_1', 'ORD_NIV_1', '01_ETO', '02_ETO', '03_ETO', '04_ETO',
       '05_ETO', '06_ETO', '07_ETO', '08_ETO', '09_ETO', '10_ETO', '11_ETO',
       '12_ETO', '01_PPT', '02_PPT', '03_PPT', '04_PPT', '05_PPT', '06_PPT',
       '07_PPT', '08_PPT', '09_PPT', '10_PPT', '11_PPT', '12_PPT',
        'kgP2O5/ha', 'N_T', 'P_T', 'K_T', '01_CWR', '02_CWR',
       '03_CWR', '04_CWR', '05_CWR', '06_CWR', '07_CWR', '08_CWR',
       '09_CWR', '10_CWR', '11_CWR', '12_CWR', '01_PPTe', '02_PPTe',
       '03_PPTe', '04_PPTe', '05_PPTe', '06_PPTe', '07_PPTe', '08_PPTe',
       '09_PPTe', '10_PPTe', '11_PPTe', '12_PPTe', '01_ID', '02_ID',
       '03_ID', '04_ID', '05_ID', '06_ID', '07_ID', '08_ID', '09_ID',
       '10_ID', '11_ID', '12_ID', 'LOSSES', '01_WE', '02_WE', '03_WE',
       '04_WE', '05_WE', '06_WE', '07_WE', '08_WE', '09_WE', '10_WE',
       '11_WE', '12_WE', 'WE_m3_ha_yr',
       'kgK2O/ha', 'kgN/ha', 
       'EF_Tier2', 'id_final', 'Kc_b_Kc1', 'Kc_b_Kc2', 'Kc_b_Kc3', 'Kc_b_Kc4',
       'Kc_b_Kc5', 'Kc_b_Kc6', 'Kc_b_Kc7', 'Kc_b_Kc8', 'Kc_b_Kc9', 'Kc_b_Kc10',
       'Kc_b_Kc11', 'Kc_b_Kc12', 'Manure', 'Manure_N', 'Manure_P2O5',
       'Agriwaste_toField', 'Agriwaste_burned', 'N_agriwaste_to_field',
       'GWP100_IPCC_N2O', 'GWP100_IPCC_CO2', 'GWP100_IPCC_CH4',
       'ME_NO3', 'ME_NO3_Reg', 'FE_PO43', 'FE_PO43_Reg', 'WC'],axis=1)
regional_impacts.head()

In [ ]:
#Fix name of Cuencas according to how they are read from the excel file with the URBAG map as shown here
pd.unique(regional_impacts['Cuencas'])

In [ ]:
###### ADAPT THE NAMES AS THEY APPEAR IN THE CELL BEFORE FOR CUENCAS IF NOT THE SAME!!!!
#There are actually six Basins in the AMB
#We place the plots in iether LLobregat or Besos basins given that only CFs are available for these two.
#Assumption 1: Plots in the Llobregat basin
#Assumption 2: Plots in the Garraf basin >> Llobregat CFs are used
#Assumption 3: Plots in rivers of the barcelona plain >> Llobregat CFs are used. These possibly drain directly to the ocean.
#Assumption 4: Plots in the Besos basin
#Assumption 5: Plots in the rivers of maresme >> Besos CFs are used. These possibly drain directly to the ocean.
#Assumption 6: Plots in the rives of the Llobregat Plain >> LLobregat CFs are used. These possibly drain directly to the ocean as well. 

cond1 = regional_impacts['Cuencas'] == 'Llobregat' 
cond2 = regional_impacts['Cuencas'] == 'Besos'

#For plots in condition 1: Midpoint regionalised impacts are
regional_impacts.loc[regional_impacts.index[cond1],'ME_NO3_midpoint']      = d_conv_emiss.loc[d_conv_emiss.index[cond1],'NO3_total'].values * NO3_CFs.loc[NO3_CFs['impact_cat']=='marine_eutroph']['CF_Llobregat_midpoint'].values
regional_impacts.loc[regional_impacts.index[cond1],'FE_PO43_midpoint']     = d_conv_emiss.loc[d_conv_emiss.index[cond1],'PO43_runoff_water_total'].values * PO43_CFs.loc[PO43_CFs['impact_cat']=='Freshwater_eutroph']['CF_Llobregat_midpoint'].values

#For plots in condition 2: Midpoint regionalised impacts are
regional_impacts.loc[regional_impacts.index[cond2],'ME_NO3_midpoint']      = d_conv_emiss.loc[d_conv_emiss.index[cond2],'NO3_total'].values * NO3_CFs.loc[NO3_CFs['impact_cat']=='marine_eutroph']['CF_Besòs_midpoint'].values
regional_impacts.loc[regional_impacts.index[cond2],'FE_PO43_midpoint']     = d_conv_emiss.loc[d_conv_emiss.index[cond2],'PO43_runoff_water_total'].values * PO43_CFs.loc[PO43_CFs['impact_cat']=='Freshwater_eutroph']['CF_Besòs_midpoint'].values


In [ ]:
#Total Midpoint regionalized impacts for:
#Marine eutrophication kg Neq/ha year
#Freshwater eutrophication kg Peq/ha year
regional_impacts['ME_midpoint']    = regional_impacts['ME_NO3_midpoint'].values
regional_impacts['FE_midpoint']    = regional_impacts['FE_PO43_midpoint'].values

In [ ]:
#For plots in condition 1: Endpoint regionalised impacts
regional_impacts.loc[regional_impacts.index[cond1],'ME_NO3_endp']      = d_conv_emiss.loc[d_conv_emiss.index[cond1],'NO3_total'].values * NO3_CFs.loc[NO3_CFs['impact_cat']=='marine_eutroph']['CF_Llobregat_endpoint'].values
regional_impacts.loc[regional_impacts.index[cond1],'FE_PO43_endp']     = d_conv_emiss.loc[d_conv_emiss.index[cond1],'PO43_runoff_water_total'].values * PO43_CFs.loc[PO43_CFs['impact_cat']=='Freshwater_eutroph']['CF_Llobregat_endpoint'].values

#For plots in condition 2: Endpoint regionalised impacts
regional_impacts.loc[regional_impacts.index[cond2],'ME_NO3_endp']      = d_conv_emiss.loc[d_conv_emiss.index[cond2],'NO3_total'].values * NO3_CFs.loc[NO3_CFs['impact_cat']=='marine_eutroph']['CF_Besòs_endpoint'].values
regional_impacts.loc[regional_impacts.index[cond2],'FE_PO43_endp']     = d_conv_emiss.loc[d_conv_emiss.index[cond2],'PO43_runoff_water_total'].values * PO43_CFs.loc[PO43_CFs['impact_cat']=='Freshwater_eutroph']['CF_Besòs_endpoint'].values


In [ ]:
#Total Endpoint regionalized impacts for:
#Marine eutrophication (potential dissapering fraction) PDF m3 yr/ha 
#Freshwater eutrophication (potential dissapering fraction) PDF m3 yr/ha
regional_impacts['ME_endpoint']  = regional_impacts['ME_NO3_endp'].values 
regional_impacts['FE_endpoint']  = regional_impacts['FE_PO43_endp'].values

In [ ]:
#For plots in condition 1: Damage regionalised impacts are
regional_impacts.loc[regional_impacts.index[cond1],'ME_NO3_dmg']      = d_conv_emiss.loc[d_conv_emiss.index[cond1],'NO3_total'].values * NO3_CFs.loc[NO3_CFs['impact_cat']=='marine_eutroph']['CF_Llobregat_damage'].values
regional_impacts.loc[regional_impacts.index[cond1],'FE_PO43_dmg']     = d_conv_emiss.loc[d_conv_emiss.index[cond1],'PO43_runoff_water_total'].values * PO43_CFs.loc[PO43_CFs['impact_cat']=='Freshwater_eutroph']['CF_Llobregat_damage'].values

#For plots in condition 2: Damage regionalised impacts are
regional_impacts.loc[regional_impacts.index[cond2],'ME_NO3_dmg']      = d_conv_emiss.loc[d_conv_emiss.index[cond2],'NO3_total'].values * NO3_CFs.loc[NO3_CFs['impact_cat']=='marine_eutroph']['CF_Besòs_damage'].values
regional_impacts.loc[regional_impacts.index[cond2],'FE_PO43_dmg']     = d_conv_emiss.loc[d_conv_emiss.index[cond2],'PO43_runoff_water_total'].values * PO43_CFs.loc[PO43_CFs['impact_cat']=='Freshwater_eutroph']['CF_Besòs_damage'].values


In [ ]:
#Total Damage regionalized impacts for:
#Marine eutrophication species yr/ha 
#Freshwater eutrophication species yr/ha
regional_impacts['ME_damage'] = regional_impacts['ME_NO3_dmg'].values
regional_impacts['FE_damage'] = regional_impacts['FE_PO43_dmg'].values

In [ ]:
regional_impacts.keys()

In [ ]:
eutrophication_impacts = regional_impacts

In [ ]:
#Marine and freshwater eutrophication impacts for Recipe and regionalized CFs 
#For direct emissions
#Midpoint ME> kg Neq/ha yr FE> kg Peq/ha yr > FROM RECIPE no emissions to air included for comparison with regionalized CFs
#Endpoint ME and FE> PDF m3 / ha
#Damage ME and FE> species yr/ha
eutrophication_impacts['ME_midpt_ReCiPe'] = d_conv_emiss['NO3_total'].values * NO3_CFs.loc[NO3_CFs['impact_cat']=='marine_eutroph']['CF_recipe'].values 
eutrophication_impacts['FE_midpt_ReCiPe'] = d_conv_emiss['PO43_runoff_water_total'].values * PO43_CFs.loc[PO43_CFs['impact_cat']=='Freshwater_eutroph']['CF_recipe'].values

In [ ]:
eutrophication_impacts.head()

In [ ]:
regional_impacts['ME_NO3_endp']

In [ ]:
URBAG_map.keys()

## Visualization

In [ ]:
eutrophication_impacts.keys()

In [ ]:
data_ME_Regional = pd.DataFrame()
data_FE_Regional = pd.DataFrame()
data_end_regional= pd.DataFrame()
data_dmg_regional= pd.DataFrame()

In [ ]:
data_ME_Regional['ME_Reg']        = eutrophication_impacts['ME_midpoint']
data_ME_Regional['ME_ReCiPe']     = eutrophication_impacts['ME_midpt_ReCiPe']

data_FE_Regional['FE_Reg']        = eutrophication_impacts['FE_midpoint']
data_FE_Regional['FE_ReCiPe']     = eutrophication_impacts['FE_midpt_ReCiPe']

data_end_regional['ME_endpoint']  = eutrophication_impacts['ME_endpoint']
data_end_regional['FE_endpoint']  = eutrophication_impacts['FE_endpoint']

data_dmg_regional['ME_damage']    = eutrophication_impacts['ME_damage']
data_dmg_regional['FE_damage']    = eutrophication_impacts['FE_damage']

In [ ]:
data_ME_Regional1 = []
for x in data_ME_Regional.keys():
    print(x)
    data_ME_Regional1.append(data_ME_Regional[x])

In [ ]:
data_FE_Regional1 = []
for x in data_FE_Regional.keys():
    print(x)
    data_FE_Regional1.append(data_FE_Regional[x])

In [ ]:
data_end_regional1 = []
for x in data_end_regional.keys():
    print(x)
    data_end_regional1.append(data_end_regional[x])

In [ ]:
data_dmg_regional1 = []
for x in data_dmg_regional.keys():
    print(x)
    data_dmg_regional1.append(data_dmg_regional[x])

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(13,13))
axs[0, 0].boxplot(data_ME_Regional1, 0, '', labels=['Total \n AMB-Specific','Total ReCiPe'], vert=True, showmeans=True)
axs[0, 0].set_title("Marine Eutrophication \n kg N$_{eq}$/ha")
axs[0, 1].boxplot(data_FE_Regional1, 0, '', labels=['Total \n AMB-Specific','Total ReCiPe'], vert=True )
axs[0, 1].set_title("Freshwater Eutrophication \n kg P$_{eq}$/ha")
axs[1, 0].boxplot(data_end_regional1, 0, '', labels=['Total ME','Total FE'], vert=True, showmeans=True)
axs[1, 0].set_title("Endpoint impacts AMB-Specific \n PDF m$^3$/ha")
axs[1, 1].boxplot(data_dmg_regional1, 0, '', labels=['Total ME','Total FE'], vert=True, showmeans=True)
axs[1, 1].set_title("Damage AMB-Specific \n species yr/ha")
#fig.savefig( 'output/' + scen + '/emissions/RegEutrophication_LCIA' + '.png')

### Saving outputs

In [ ]:
%store data_foreground_1
%store eutrophication_impacts
%store URBAG_map

In [ ]:
#Eutrophication local impacts with regionalized CFs for nitrate and phosphate emissions only kg Peq/ha and kg Neq/ha
eutrophication_impacts.to_csv("output/" + scen + "/LCIA/regionalised_eutrophication.csv")

In [ ]:
#Total foreground impacts
#imp_per_ha in kg impact per ha year
#imp_yr in ton per year
data_foreground.to_csv("output/" + scen + "/LCIA/foregroundImpacts/Map_foreground_impacts.csv")

In [ ]:
#Total foreground impacts in ton per ha yr
data_foreground_1.to_csv("output/" + scen + "/LCIA/foregroundImpacts/Total_foreground_impacts.csv")